In [1]:
import pandas as pd

In [2]:
# Read raw structural data and store it in a DataFrame
import_data = pd.read_csv('/Users/lutz/Documents/german_federal_elections/data/btw21_structual_data_raw.csv', sep=';')

# Extract relevant rows and set column names
structural_data = import_data.iloc[7:, :]
structural_data.columns = structural_data.iloc[0]
structural_data = structural_data.drop(structural_data.index[0]).reset_index(drop=True)

# Remove rows where the third column has the value 'Land insgesamt'
rows_to_remove = structural_data[structural_data.iloc[:, 2] == 'Land insgesamt']
structural_data = structural_data.drop(rows_to_remove.index)

# Remove unnecessary rows and columns, and reset the index
structural_data.drop([315], inplace=True)
structural_data.drop(['Fußnoten'], inplace=True, axis=1)
structural_data.reset_index(inplace=True, drop=True)

# Assign proper district numbers
structural_data['Wahlkreis-Nr.'] = list(range(1, 300))


# Define a dictionary with the original column names as keys and shortened names as values
column_name_mapping = {
    'Land':'state',
    'Wahlkreis-Nr.':'district_number',
    'Wahlkreis-Name': 'District_Name',
    'Gemeinden am 31.12.2019 (Anzahl)': 'Num_Municipalities',
    'Fläche am 31.12.2019 (km²)': 'Area_km2', #raus
    'Bevölkerung am 31.12.2019 - Insgesamt (in 1000)': 'Total_Population',#raus
    'Bevölkerung am 31.12.2019 - Deutsche (in 1000)': 'German_Population_Perc',#raus
    'Bevölkerung am 31.12.2019 - Ausländer/-innen (%)': 'Foreign_Population_Perc',
    'Bevölkerungsdichte am 31.12.2019 (EW je km²)': 'Population_Density (IH per km2)',
    'Zu- (+) bzw. Abnahme (-) der Bevölkerung 2019 - Geburtensaldo (je 1000 EW)': 'Birth_Balance (per 1000 IH)',
    'Zu- (+) bzw. Abnahme (-) der Bevölkerung 2019 - Wanderungssaldo (je 1000 EW)': 'Migration_Balance (per 1000 IH)',
    'Alter von ... bis ... Jahren am 31.12.2019 - unter 18 (%)': 'Age_Under_18_Perc',
    'Alter von ... bis ... Jahren am 31.12.2019 - 18-24 (%)': 'Age_18_24_Perc',
    'Alter von ... bis ... Jahren am 31.12.2019 - 25-34 (%)': 'Age_25_34_Perc',
    'Alter von ... bis ... Jahren am 31.12.2019 - 35-59 (%)': 'Age_35_59_Perc',
    'Alter von ... bis ... Jahren am 31.12.2019 - 60-74 (%)': 'Age_60_74_Perc',
    'Alter von ... bis ... Jahren am 31.12.2019 - 75 und mehr (%)': 'Age_75_Up_Perc',
    'Bodenfläche nach Art der tatsächlichen Nutzung am 31.12.2019 - Siedlung und Verkehr (%)': 'Land_Use_Settlement_Perc',
    'Bodenfläche nach Art der tatsächlichen Nutzung am 31.12.2019 - Vegetation und Gewässer (%)': 'Land_Use_Vegetation_Perc',
    'Fertiggestellte Wohnungen 2019 (je 1000 EW)': 'Completed_Housing (per 1000 IH)',
    'Bestand an Wohnungen am 31.12.2019 - insgesamt (je 1000 EW)': 'Housing_Stock (per 1000 IH)',
    'Wohnfläche am 31.12.2019 (je Wohnung)': 'Living_Space_per_Housing',
    'Wohnfläche am 31.12.2019 (je EW)': 'Living_Space_per_Person',
    'PKW-Bestand am 01.01.2020 - PKW insgesamt (je 1000 EW)': 'Total_Cars (per 1000 IH)',
    'PKW-Bestand am 01.01.2020 - PKW mit Elektro- oder Hybrid-Antrieb (%)': 'Electric_Hybrid_Cars_Perc',
    'Unternehmensregister 2018 - Unternehmen insgesamt (je 1000 EW)': 'Total_Enterprises (per 1000 IH)',
    'Unternehmensregister 2018 - Handwerksunternehmen (je 1000 EW)': 'Craft_Enterprises (per 1000 IH)',
    'Schulabgänger/-innen beruflicher Schulen 2019': 'Vocational_School_Graduates',
    'Schulabgänger/-innen allgemeinbildender Schulen 2019 - insgesamt ohne Externe (je 1000 EW)': 'General_School_Graduates (per 1000 IH)',
    'Schulabgänger/-innen allgemeinbildender Schulen 2019 - ohne Hauptschulabschluss (%)': 'No_School_Diploma_Perc',
    'Schulabgänger/-innen allgemeinbildender Schulen 2019 - mit Hauptschulabschluss (%)': 'Hauptschulabschluss_Perc',
    'Schulabgänger/-innen allgemeinbildender Schulen 2019 - mit mittlerem Schulabschluss (%)': 'Mittlerer_Schulabschluss_Perc',
    'Schulabgänger/-innen allgemeinblldender Schulen 2019 - mit allgemeiner und Fachhochschulreife (%)': 'Higher_Education_Diploma_Perc',
    'Kindertagesbetreuung am 01.03.2020 - Betreute Kinder unter 3 Jahre (Betreuungsquote)': 'Childcare_Under_3_Perc',
    'Kindertagesbetreuung am 01.03.2020 - Betreute Kinder 3 bis unter 6 Jahre (Betreuungsquote)': 'Childcare_3_to_6_Perc',
    'Verfügbares Einkommen der privaten Haushalte 2018 (EUR je EW)': 'Disposable_Income_per_Person',
    'Bruttoinlandsprodukt 2018 (EUR je EW)': 'GDP_per_Person',
    'Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - insgesamt (je 1000 EW)': 'Employees_Total (per 1000 IH)',
    'Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Land- und Forstwirtschaft, Fischerei (%)': 'Employees_Agriculture_Perc',
    'Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Produzierendes Gewerbe (%)': 'Employees_Manufacturing_Perc',
    'Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Handel, Gastgewerbe, Verkehr (%)': 'Employees_Trade_Perc',
    'Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Öffentliche und private Dienstleister (%)': 'Employees_Service_Perc',
    'Sozialversicherungspflichtig Beschäftigte am 30.06.2020 - Übrige Dienstleister und "ohne Angabe" (%)': 'Employees_Other_Perc',
    'Empfänger/-innen von Leistungen nach SGB II  Oktober 2020 -  insgesamt (je 1000 EW)': 'SGB_II_Recipients_Total (per 1000 IH)',
    'Empfänger/-innen von Leistungen nach SGB II  Oktober 2020 -  nicht erwerbsfähige Hilfebedürftige (%)': 'SGB_II_Non_Employable_Perc',
    'Empfänger/-innen von Leistungen nach SGB II  Oktober 2020 -  Ausländer/-innen (%)': 'SGB_II_Foreigner_Perc',
    'Arbeitslosenquote Februar 2021 - insgesamt': 'Unemployment_Rate_Total',
    'Arbeitslosenquote Februar 2021 - Männer': 'Unemployment_Rate_Men',
    'Arbeitslosenquote Februar 2021 - Frauen': 'Unemployment_Rate_Women',
    'Arbeitslosenquote Februar 2021 - 15 bis 24 Jahre': 'Unemployment_Rate_Age_15_24',
    'Arbeitslosenquote Februar 2021 - 55 bis 64 Jahre': 'Unemployment_Rate_Age_55_64'
}

# Rename the columns in the DataFrame using the mapping
structural_data.rename(columns=column_name_mapping, inplace=True)


def change_unit(x):
    if '(per 1000 IH)' in x.name:
        x=x.astype('float')/10  
    else:
        x=x.astype('float')      
    return x

# Clean the data by removing unwanted characters and converting data types
slice_1 = structural_data.iloc[:, 3:].apply(lambda x: x.str.replace('.', '').str.replace(',', '.').astype('float'))
slice_1 = slice_1.apply(change_unit)
slice_2 = structural_data.iloc[:, :3]
structural_data = pd.concat([slice_2, slice_1], axis=1)


# Specific string to search for
search_string = " (per 1000 IH)"

# Create a dictionary mapping old column names to new column names
column_name_mapping = {
    col: col.replace(search_string, "_Perc") if search_string in col else col
    for col in structural_data.columns
}

# Rename the columns using the mapping
structural_data = structural_data.rename(columns=column_name_mapping)


# Convert population-related data to appropriate units
structural_data['Total_Population'] = structural_data['Total_Population'] * 1000
structural_data['German_Population_Perc'] = 100 - structural_data['Foreign_Population_Perc']
structural_data['Vocational_School_Graduates'] = structural_data['Vocational_School_Graduates'] / structural_data['Total_Population'] * 100

# Save the cleaned structural data to a CSV file
#structural_data.to_csv('/Users/Lutz/Documents/german_federal_elections/output/structural_data.csv')

structural_data

/var/folders/k4/6z9w72t12vlgl457wsdgcr5w0000gn/T/ipykernel_96309/3087650765.py:89: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  slice_1 = structural_data.iloc[:, 3:].apply(lambda x: x.str.replace('.', '').str.replace(',', '.').astype('float'))


7,state,district_number,District_Name,Num_Municipalities,Area_km2,Total_Population,German_Population_Perc,Foreign_Population_Perc,Population_Density (IH per km2),Birth_Balance_Perc,...,Employees_Service_Perc,Employees_Other_Perc,SGB_II_Recipients_Total_Perc,SGB_II_Non_Employable_Perc,SGB_II_Foreigner_Perc,Unemployment_Rate_Total,Unemployment_Rate_Men,Unemployment_Rate_Women,Unemployment_Rate_Age_15_24,Unemployment_Rate_Age_55_64
0,Schleswig-Holstein,1,Flensburg – Schleswig,126.0,2124.3,291300.0,91.6,8.4,137.1,-0.27,...,15.4,36.7,7.68,26.1,28.2,7.0,7.7,6.2,5.9,7.6
1,Schleswig-Holstein,2,Nordfriesland – Dithmarschen Nord,197.0,2777.9,235000.0,92.9,7.1,84.6,-0.52,...,11.8,32.7,5.91,26.1,23.8,6.5,6.9,5.9,5.4,7.2
2,Schleswig-Holstein,3,Steinburg – Dithmarschen Süd,178.0,2000.0,221000.0,93.5,6.5,110.5,-0.53,...,15.1,32.0,7.06,26.3,29.3,6.4,6.8,6.1,7.0,6.4
3,Schleswig-Holstein,4,Rendsburg-Eckernförde,163.0,2165.4,252100.0,94.5,5.5,116.4,-0.38,...,17.0,33.9,5.28,29.6,33.2,4.8,5.2,4.3,4.7,5.2
4,Schleswig-Holstein,5,Kiel,3.0,143.0,268800.0,88.6,11.4,1879.2,-0.08,...,24.8,38.9,12.52,26.8,35.1,8.4,9.2,7.4,5.1,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,Baden-Württemberg,295,Zollernalb – Sigmaringen,40.0,1656.3,248800.0,87.7,12.3,150.2,-0.21,...,9.0,24.0,2.76,25.5,31.0,4.1,4.2,3.9,3.9,4.8
295,Saarland,296,Saarbrücken,6.0,325.7,271400.0,83.6,16.4,833.3,-0.40,...,25.1,32.6,13.18,26.9,42.6,10.3,11.4,9.2,8.1,9.2
296,Saarland,297,Saarlouis,18.0,893.2,262400.0,89.1,10.9,293.8,-0.58,...,13.8,25.9,5.50,26.3,40.4,5.7,6.2,5.0,3.7,5.9
297,Saarland,298,St. Wendel,16.0,802.0,212600.0,93.5,6.5,265.1,-0.65,...,14.6,30.6,6.96,27.6,40.6,6.3,6.7,5.8,5.3,6.1


In [ ]:
# Import the raw election data
import_data_election=pd.read_csv('/Users/lutz/Documents/german_federal_elections/data/election_data_raw.csv',header=None, sep=';')

# Extract relevant columns
election_data=import_data_election.iloc[2:,:51]

# Remove columns containing the string 'Vorperiode'
string_to_search = "Vorperiode"
cols_to_remove = []

# Iterate through the columns and check if the string is present in any of the column values
for col in election_data.columns:
    if election_data[col].apply(lambda x: string_to_search in str(x)).any():
        cols_to_remove.append(col)

# Drop the columns containing the specified string
election_data = election_data.drop(columns=cols_to_remove)

# Merge first three rows and set as column headers, remove unnecessary characters
merged_row = election_data.iloc[:3,:].apply(lambda x: ', '.join(x.astype(str)), axis=0)
merged_row=merged_row.str.replace('nan','').str.replace(' ','').str.replace(',','')
election_data.columns=list(merged_row)

# Drop irrelevant rows
election_data.drop([2,3,4], inplace=True)
election_data.drop([335,336], inplace=True)

# Remove rows with '99' in the third column
rows_to_remove = election_data[election_data.iloc[:, 2] == '99']
election_data = election_data.drop(rows_to_remove.index)

# Clean up the dataset
election_data.dropna(how='all', inplace=True)
election_data.reset_index(inplace=True, drop=True)

# Rename and renumber the first column
election_data.rename(columns={'Nr':'Wahlkreis-Nr.'}, inplace=True)
election_data['Wahlkreis-Nr.']=list(range(1,300))

# Map old column names to new column names
column_name_mapping = {
    'Wahlkreis-Nr.': 'District_Number',
    'Gebiet': 'Area',
    'gehörtzu': 'Belongs_To',
    'WahlberechtigteErststimmenEndgültig': 'Eligible_First_Votes_Final',
    'WahlberechtigteZweitstimmenEndgültig': 'Eligible_Second_Votes_Final',
    'WählendeErststimmenEndgültig': 'Voters_First_Votes_Final',
    'WählendeZweitstimmenEndgültig': 'Voters_Second_Votes_Final',
    'UngültigeStimmenErststimmenEndgültig': 'Invalid_First_Votes_Final',
    'UngültigeStimmenZweitstimmenEndgültig': 'Invalid_Second_Votes_Final',
    'GültigeStimmenErststimmenEndgültig': 'Valid_First_Votes_Final',
    'GültigeStimmenZweitstimmenEndgültig': 'Valid_Second_Votes_Final',
    'ChristlichDemokratischeUnionDeutschlandsErststimmenEndgültig': 'CDU_First_Votes_Final',
    'ChristlichDemokratischeUnionDeutschlandsZweitstimmenEndgültig': 'CDU_Second_Votes_Final',
    'SozialdemokratischeParteiDeutschlandsErststimmenEndgültig': 'SPD_First_Votes_Final',
    'SozialdemokratischeParteiDeutschlandsZweitstimmenEndgültig': 'SPD_Second_Votes_Final',
    'AlternativefürDeutschlandErststimmenEndgültig': 'AfD_First_Votes_Final',
    'AlternativefürDeutschlandZweitstimmenEndgültig': 'AfD_Second_Votes_Final',
    'FreieDemokratischeParteiErststimmenEndgültig': 'FDP_First_Votes_Final',
    'FreieDemokratischeParteiZweitstimmenEndgültig': 'FDP_Second_Votes_Final',
    'DIELINKEErststimmenEndgültig': 'Left_First_Votes_Final',
    'DIELINKEZweitstimmenEndgültig': 'Left_Second_Votes_Final',
    'BÜNDNIS90/DIEGRÜNENErststimmenEndgültig': 'Greens_First_Votes_Final',
    'BÜNDNIS90/DIEGRÜNENZweitstimmenEndgültig': 'Greens_Second_Votes_Final',
    'Christlich-SozialeUnioninBayerne.V.ErststimmenEndgültig': 'CSU_First_Votes_Final',
    'Christlich-SozialeUnioninBayerne.V.ZweitstimmenEndgültig': 'CSU_Second_Votes_Final',
    'FREIEWÄHLERErststimmenEndgültig': 'FW_First_Votes_Final',
    'FREIEWÄHLERZweitstimmenEndgültig': 'FW_Second_Votes_Final'
}

# Rename the columns in the DataFrame using the mapping
election_data.rename(columns=column_name_mapping, inplace=True)

# Replace '.' with '' and ',' with '.' for easier conversion to float
slice_1=election_data.iloc[:,3:].apply(lambda x: x.str.replace('.','').str.replace(',','.').astype('float'))
slice_2=election_data.iloc[:,:3]

# Concatenate slices to create a cleaned dataset
election_data=pd.concat([slice_2, slice_1], axis=1)

# Define column groups for first and second votes
first_votes_columns = [
    'Eligible_First_Votes_Final',
    'Voters_First_Votes_Final',
    'Invalid_First_Votes_Final',
    'Valid_First_Votes_Final',
    'CDU_First_Votes_Final',
    'SPD_First_Votes_Final',
    'AfD_First_Votes_Final',
    'FDP_First_Votes_Final',
    'Left_First_Votes_Final',
    'Greens_First_Votes_Final',
    'CSU_First_Votes_Final',
    'FW_First_Votes_Final'
]

second_votes_columns = [
    'Eligible_Second_Votes_Final',
    'Voters_Second_Votes_Final',
    'Invalid_Second_Votes_Final',
    'Valid_Second_Votes_Final',
    'CDU_Second_Votes_Final',
    'SPD_Second_Votes_Final',
    'AfD_Second_Votes_Final',
    'FDP_Second_Votes_Final',
    'Left_Second_Votes_Final',
    'Greens_Second_Votes_Final',
    'CSU_Second_Votes_Final',
    'FW_Second_Votes_Final'
]
other_columns=['District_Number', 'Area', 'Belongs_To']

# Separate first and second votes data
first_votes_data=election_data[other_columns+first_votes_columns]
second_votes_data=election_data[other_columns+second_votes_columns]

# Calculate percentages for first votes
slice_1=first_votes_data.iloc[:,:7]
slice_2=first_votes_data.iloc[:,7:].apply(lambda x: x/first_votes_data['Valid_First_Votes_Final']*100 )
first_votes_data=pd.concat([slice_1,slice_2], axis=1)

# Calculate percentages for second votes
slice_1=second_votes_data.iloc[:,:7]
slice_2=second_votes_data.iloc[:,7:].apply(lambda x: x/second_votes_data['Valid_Second_Votes_Final']*100 )
second_votes_data=pd.concat([slice_1,slice_2], axis=1)

# Save cleaned data to CSV files
election_data.to_csv('/Users/Lutz/Documents/german_federal_elections/output/election_data_complete.csv')
first_votes_data.to_csv('/Users/Lutz/Documents/german_federal_elections/output/first_votes_data.csv')
second_votes_data.to_csv('/Users/Lutz/Documents/german_federal_elections/output/second_votes_data.csv')
